In [1]:
from nltk.corpus import brown
import torch
from torch import LongTensor
from torch.nn import Embedding, LSTM
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from gensim.models import KeyedVectors

In [2]:
index_to_word = list(set(brown.words()))
word_to_index = dict()
for index, word in enumerate(index_to_word):
    word_to_index[word] = index+2

In [3]:
vectors = KeyedVectors.load_word2vec_format('/home/development/kevin/../girish/word-embeddings/GoogleNews-vectors-negative300.bin', binary=True)
E = np.zeros((len(word_to_index)+2, 300))
avg_vec = np.mean(vectors.syn0, axis=0)
for word in word_to_index:
    if word in vectors:
        E[word_to_index[word]] = vectors[word]
    else:
        E[word_to_index[word]] = avg_vec
E = torch.from_numpy(E)
# E[1] = avg_vec

/home/development/kevin/installations/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  app.launch_new_instance()


In [59]:
list_of_sents_trg = []
for sent in brown.sents():
    if len(sent) > 30:
        continue
    single_sent=[]
    for word in sent:
        single_sent.append(word_to_index[word])
    list_of_sents_trg.append(single_sent)

In [60]:
list_of_sents_src = []
f= open("shuffle.txt", "r")
for s in f:
    single_sent=[]
    sent = s.split(' ')[:-1]
    if len(sent) > 30:
        continue
    for word in sent:
        single_sent.append(word_to_index[word])
    list_of_sents_src.append(single_sent)

In [61]:
for w in list_of_sents_src[0]:
    print(index_to_word[w])
print('-----------')

    

Hidden
$8.5
resistive
thin
transcript
scornfully
1.00
'30s
pantheon
lifeboat
Victory
Inter-American
ante
Hausman's
Shakya
murmured
whit
Kedzie
watchful
centerline
1601
presumptions
minimize
Petermann
slinging
-----------


In [62]:
# max1 = 0
# for sent in list_of_sents_src:
#     if(len(sent)>= max1):
#         max1= len(sent)
# print(max1)
# print(len(list_of_sents_src))

In [63]:
def data_gen1(list_of_sents_src, list_of_sents_trg, num_words=11, batch_size=16, num_batches=100, length=30, pad_index=0, sos_index=1):
    """Generate random data for a src-tgt copy task."""
#     print('see hte length is ',len(list_of_sents_src[0]))#if less then or = 25 catch list_of_sents_src
    index=0
    for i in range(num_batches):
        temp=[]
        for sent in list_of_sents_src[index:index+(batch_size)]:
#             temp.append([1]+sent+np.zeros((length-len(sent)-1), dtype=int).tolist())
            temp.append([1]+sent)
        
        seq_lengths = LongTensor(list(map(len, temp)))
        seq_tensor = Variable(torch.zeros((len(temp), seq_lengths.max()))).long()
        for idx, (seq, seqlen) in enumerate(zip(temp, seq_lengths)):
            seq_tensor[idx, :seqlen] = LongTensor(seq)
        seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
        #print('Perm Index for src', perm_idx)
        seq_tensor = seq_tensor[perm_idx]
#         data = torch.from_numpy(np.array(temp))
        data = torch.from_numpy(np.array(seq_tensor))
        
        data = data.cuda(device=1) if USE_CUDA else data
        src = data[:, 1:]
        
        temp=[]
        for sent in list_of_sents_trg[index:index+(batch_size)]:
#             temp.append(sent+np.zeros((length-len(sent)), dtype=int).tolist())
            temp.append([1]+sent)
        
#         input_list = temp[31]
#         input_words = []
#         for w in input_list:
#             input_words.append(index_to_word[w])
#         print(input_words,len(input_words))
        
        seq_lengths = LongTensor(list(map(len, temp)))
        seq_tensor = Variable(torch.zeros((len(temp), seq_lengths.max()))).long()
        for idx, (seq, seqlen) in enumerate(zip(temp, seq_lengths)):
            seq_tensor[idx, :seqlen] = LongTensor(seq)
        seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
        #print('Perm Index for trg', perm_idx)
        seq_tensor = seq_tensor[perm_idx]
#         data = torch.from_numpy(np.array(temp))
        data = torch.from_numpy(np.array(seq_tensor))
        trg = data
#         print(src.size(), trg.size())
#         print(length-1, data.size(1)-1)
#         src_lengths = [length-1] * batch_size
#         trg_lengths = [length] * batch_size
        src_lengths = [src.size(1)] * batch_size
        trg_lengths = [trg.size(1)] * batch_size
        index = batch_size
        yield Batch((src, src_lengths), (trg, trg_lengths), pad_index=pad_index)

In [64]:
length = len(list_of_sents_src)
# list_of_sents_src_train, list_of_sents_src_test = list_of_sents_src[:int(0.7*length)], list_of_sents_src[int(0.7*length):]
# list_of_sents_trg_train, list_of_sents_trg_test = list_of_sents_trg[:int(0.7*length)], list_of_sents_trg[int(0.7*length):]

# list_of_sents_src_train, list_of_sents_src_test = list_of_sents_src[int(0.3*length):], list_of_sents_src[:int(0.3*length)]
# list_of_sents_trg_train, list_of_sents_trg_test = list_of_sents_trg[int(0.3*length):], list_of_sents_trg[:int(0.3*length)]

list_of_sents_src_train, list_of_sents_src_test = list_of_sents_src[int(0.15*length):int(0.85*length)], list_of_sents_src[:int(0.15*length)]+list_of_sents_src[int(0.85*length):]
list_of_sents_trg_train, list_of_sents_trg_test = list_of_sents_trg[int(0.15*length):int(0.85*length)], list_of_sents_trg[:int(0.15*length)]+list_of_sents_src[int(0.85*length):]

# The Annotated Encoder-Decoder with Attention

Recently, Alexander Rush wrote a blog post called [The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html), describing the Transformer model from the paper [Attention is All You Need](https://arxiv.org/abs/1706.03762). This post can be seen as a **prequel** to that: *we will implement an Encoder-Decoder with Attention* using (Gated) Recurrent Neural Networks, very closely following the original attention-based neural machine translation paper ["Neural Machine Translation by Jointly Learning to Align and Translate"](https://arxiv.org/abs/1409.0473) of Bahdanau et al. (2015). 

The idea is that going through both blog posts will make you familiar with two very influential sequence-to-sequence architectures. If you have any comments or suggestions, please let me know on Twitter [@joostbastings](https://twitter.com/joostbastings) or e-mail me at *firstname dot lastname @ gmail dot com*.

# Model Architecture

We will model the probability $p(Y\mid X)$ of a target sequence $Y=(y_1, \dots, y_{N})$ given a source sequence $X=(x_1, \dots, x_M)$ directly with a neural network: an Encoder-Decoder.

<img src="images/bahdanau.png" width="636">

#### Encoder 

The encoder reads in the source sentence (*at the bottom of the figure*) and produces a sequence of hidden states $\mathbf{h}_1, \dots, \mathbf{h}_M$, one for each source word. These states should capture the meaning of a word in its context of the given sentence.

We will use a bi-directional recurrent neural network (Bi-RNN) as the encoder; a Bi-GRU in particular.

First of all we **embed** the source words. 
We simply look up the **word embedding** for each word in a (randomly initialized) lookup table.
We will denote the word embedding for word $i$ in a given sentence with $\mathbf{x}_i$.
By embedding words, our model may exploit the fact that certain words (e.g. *cat* and *dog*) are semantically similar, and can be processed in a similar way.

Now, how do we get hidden states $\mathbf{h}_1, \dots, \mathbf{h}_M$? A forward GRU reads the source sentence left-to-right, while a backward GRU reads it right-to-left.
Each of them follows a simple recursive formula: 
$$\mathbf{h}_j = \text{GRU}( \mathbf{x}_j , \mathbf{h}_{j - 1} )$$
i.e. we obtain the next state from the previous state and the current input word embedding.

The hidden state of the forward GRU at time step $j$ will know what words **precede** the word at that time step, but it doesn't know what words will follow. In contrast, the backward GRU will only know what words **follow** the word at time step $j$. By **concatenating** those two hidden states (*shown in blue in the figure*), we get $\mathbf{h}_j$, which captures word $j$ in its full sentence context.


#### Decoder 

The decoder (*at the top of the figure*) is a GRU with hidden state $\mathbf{s_i}$. It follows a similar formula to the encoder, but takes one extra input $\mathbf{c}_{i}$ (*shown in yellow*).

$$\mathbf{s}_{i} = f( \mathbf{s}_{i - 1}, \mathbf{y}_{i - 1}, \mathbf{c}_i )$$

Here, $\mathbf{y}_{i - 1}$ is the previously generated target word (*not shown*).

At each time step, an **attention mechanism** dynamically selects that part of the source sentence that is most relevant for predicting the current target word. It does so by comparing the last decoder state with each source hidden state. The result is a context vector $\mathbf{c_i}$ (*shown in yellow*).
Later the attention mechanism is explained in more detail.

After computing the decoder state $\mathbf{s}_i$, a non-linear function $g$ (which applies a [softmax](https://en.wikipedia.org/wiki/Softmax_function)) gives us the probability of the target word $y_i$ for this time step:

$$ p(y_i \mid y_{<i}, x_1^M) = g(\mathbf{s}_i, \mathbf{c}_i, \mathbf{y}_{i - 1})$$

Because $g$ applies a softmax, it provides a vector the size of the output vocabulary that sums to 1.0: it is a distribution over all target words. During test time, we would select the word with the highest probability for our translation.

Now, for optimization, a [cross-entropy loss](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy) is used to maximize the probability of selecting the correct word at this time step. All parameters (including word embeddings) are then updated to maximize this probability.



# Prelims

This tutorial requires **PyTorch >= 0.4.1** and was tested with **Python 3.6**.  

Make sure you have those versions, and install the packages below if you don't have them yet.

In [65]:
#!pip install torch numpy matplotlib sacrebleu

In [66]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from IPython.core.debugger import set_trace

# we will use CUDA if it is available
USE_CUDA = torch.cuda.is_available()
DEVICE=torch.device('cuda:0') # or set to 'cpu'
print("CUDA:", USE_CUDA)
print(DEVICE)

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

CUDA: True
cuda:0


# Let's start coding!

## Model class

Our base model class `EncoderDecoder` is very similar to the one in *The Annotated Transformer*.

One difference is that our encoder also returns its final states (`encoder_final` below), which is used to initialize the decoder RNN. We also provide the sequence lengths as the RNNs require those.

In [67]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, trg_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.src_embed.weight.data.copy_(E)
        self.trg_embed = trg_embed
        self.trg_embed.weight.data.copy_(E)
        self.generator = generator
        
    def forward(self, src, trg, src_mask, trg_mask, src_lengths, trg_lengths):
        """Take in and process masked src and target sequences."""
        encoder_hidden, encoder_final = self.encode(src, src_mask, src_lengths)
        return self.decode(encoder_hidden, encoder_final, src_mask, trg, trg_mask)
    
    def encode(self, src, src_mask, src_lengths):
        return self.encoder(self.src_embed(src), src_mask, src_lengths)
    
    def decode(self, encoder_hidden, encoder_final, src_mask, trg, trg_mask,
               decoder_hidden=None):
        return self.decoder(self.trg_embed(trg), encoder_hidden, encoder_final,
                            src_mask, trg_mask, hidden=decoder_hidden)

To keep things easy we also keep the `Generator` class the same. 
It simply projects the pre-output layer ($x$ in the `forward` function below) to obtain the output layer, so that the final dimension is the target vocabulary size.

In [68]:
class Generator(nn.Module):
    """Define standard linear + softmax generation step."""
    def __init__(self, hidden_size, vocab_size):
        super(Generator, self).__init__()
        self.proj = nn.Linear(hidden_size, vocab_size, bias=False)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

## Encoder

Our encoder is a bi-directional GRU. 

Because we want to process multiple sentences at the same time for speed reasons (it is more effcient on GPU), we need to support **mini-batches**. Sentences in a mini-batch may have different lengths, which means that the RNN needs to unroll further for certain sentences while it might already have finished for others:

```
Example: mini-batch with 3 source sentences of different lengths (7, 5, and 3).
End-of-sequence is marked with a "3" here, and padding positions with "1".

+---------------+
| 4 5 9 8 7 8 3 |
+---------------+
| 5 4 8 7 3 1 1 |
+---------------+
| 5 8 3 1 1 1 1 |
+---------------+
```
You can see that, when computing hidden states for this mini-batch, for sentence #2 and #3 we will need to stop updating the hidden state after we have encountered "3". We don't want to incorporate the padding values (1s).

Luckily, PyTorch has convenient helper functions called `pack_padded_sequence` and `pad_packed_sequence`.
These functions take care of masking and padding, so that the resulting word representations are simply zeros after a sentence stops.

The code below reads in a source sentence (a sequence of word embeddings) and produces the hidden states.
It also returns a final vector, a summary of the complete sentence, by concatenating the first and the last hidden states (they have both seen the whole sentence, each in a different direction). We will use the final vector to initialize the decoder.

In [69]:
class Encoder(nn.Module):
    """Encodes a sequence of word embeddings"""
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, 
                          batch_first=True, bidirectional=True, dropout=dropout)
        
    def forward(self, x, mask, lengths):
        """
        Applies a bidirectional GRU to sequence of embeddings x.
        The input mini-batch x needs to be sorted by length.
        x should have dimensions [batch, time, dim].
        """
        packed = pack_padded_sequence(x, lengths, batch_first=True)
        output, final = self.rnn(packed)
        output, _ = pad_packed_sequence(output, batch_first=True)

        # we need to manually concatenate the final states for both directions
        fwd_final = final[0:final.size(0):2]
        bwd_final = final[1:final.size(0):2]
        final = torch.cat([fwd_final, bwd_final], dim=2)  # [num_layers, batch, 2*dim]

#         print('Inside encoder forward', output.size(), final.size())
        return output, final

### Decoder

The decoder is a conditional GRU. Rather than starting with an empty state like the encoder, its initial hidden state results from a projection of the encoder final vector. 

#### Training
In `forward` you can find a for-loop that computes the decoder hidden states one time step at a time. 
Note that, during training, we know exactly what the target words should be! (They are in `trg_embed`.) This means that we are not even checking here what the prediction is! We simply feed the correct previous target word embedding to the GRU at each time step. This is called teacher forcing.

The `forward` function returns all decoder hidden states and pre-output vectors. Elsewhere these are used to compute the loss, after which the parameters are updated.

#### Prediction
For prediction time, for forward function is only used for a single time step. After predicting a word from the returned pre-output vector, we can call it again, supplying it the word embedding of the previously predicted word and the last state.

In [70]:
class Decoder(nn.Module):
    """A conditional RNN decoder with attention."""
    
    def __init__(self, emb_size, hidden_size, attention, num_layers=1, dropout=0.5,
                 bridge=True):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.attention = attention
        self.dropout = dropout
                 
        self.rnn = nn.GRU(emb_size + 2*hidden_size, hidden_size, num_layers,
                          batch_first=True, dropout=dropout)
                 
        # to initialize from the final encoder state
        self.bridge = nn.Linear(2*hidden_size, hidden_size, bias=True) if bridge else None

        self.dropout_layer = nn.Dropout(p=dropout)
        self.pre_output_layer = nn.Linear(hidden_size + 2*hidden_size + emb_size,
                                          hidden_size, bias=False)
        
    def forward_step(self, prev_embed, encoder_hidden, src_mask, proj_key, hidden):
        """Perform a single decoder step (1 word)"""

        # compute context vector using attention mechanism
#         print(hidden.size())
        query = hidden[-1].unsqueeze(1)  # [#layers, B, D] -> [B, 1, D]
#         print('Inside forward step')
#         print(query.size())
#         print(proj_key.size())
        context, attn_probs = self.attention(
            query=query, proj_key=proj_key,
            value=encoder_hidden, mask=src_mask)

        # update rnn hidden state
#         print(prev_embed.size(), context.size())
        rnn_input = torch.cat([prev_embed, context], dim=2)
        output, hidden = self.rnn(rnn_input, hidden)
        
        pre_output = torch.cat([prev_embed, output, context], dim=2)
        pre_output = self.dropout_layer(pre_output)
        pre_output = self.pre_output_layer(pre_output)

        return output, hidden, pre_output
    
    def forward(self, trg_embed, encoder_hidden, encoder_final, 
                src_mask, trg_mask, hidden=None, max_len=None):
        """Unroll the decoder one step at a time."""
        
#         print('Inside forward: ', trg_embed.size())
        # the maximum number of steps to unroll the RNN
        if max_len is None:
            max_len = trg_mask.size(-1)

        # initialize decoder hidden state
        if hidden is None:
            hidden = self.init_hidden(encoder_final)
        
        # pre-compute projected encoder hidden states
        # (the "keys" for the attention mechanism)
        # this is only done for efficiency
#         print('Inside forward')
#         print(encoder_hidden.size())
        proj_key = self.attention.key_layer(encoder_hidden)
#         print(proj_key.size())
        # here we store all intermediate hidden states and pre-output vectors
        decoder_states = []
        pre_output_vectors = []
        
        # unroll the decoder RNN for max_len steps
        for i in range(max_len):
#             print('Inside unroll')
            prev_embed = trg_embed[:, i].unsqueeze(1)
            output, hidden, pre_output = self.forward_step(
              prev_embed, encoder_hidden, src_mask, proj_key, hidden)
            decoder_states.append(output)
            pre_output_vectors.append(pre_output)

        decoder_states = torch.cat(decoder_states, dim=1)
        pre_output_vectors = torch.cat(pre_output_vectors, dim=1)
        return decoder_states, hidden, pre_output_vectors  # [B, N, D]

    def init_hidden(self, encoder_final):
        """Returns the initial decoder state,
        conditioned on the final encoder state."""

        if encoder_final is None:
            return None  # start with zeros

        return torch.tanh(self.bridge(encoder_final))            


### Attention                                                                                                                                                                               

At every time step, the decoder has access to *all* source word representations $\mathbf{h}_1, \dots, \mathbf{h}_M$. 
An attention mechanism allows the model to focus on the currently most relevant part of the source sentence.
The state of the decoder is represented by GRU hidden state $\mathbf{s}_i$.
So if we want to know which source word representation(s) $\mathbf{h}_j$ are most relevant, we will need to define a function that takes those two things as input.

Here we use the MLP-based, additive attention that was used in Bahdanau et al.:

<img src="images/attention.png" width="280">


We apply an MLP with tanh-activation to both the current decoder state $\bf s_i$ (the *query*) and each encoder state $\bf h_j$ (the *key*), and then project this to a single value (i.e. a scalar) to get the *attention energy* $e_{ij}$. 

Once all energies are computed, they are normalized by a softmax so that they sum to one: 

$$ \alpha_{ij} = \text{softmax}(\mathbf{e}_i)[j] $$

$$\sum_j \alpha_{ij} = 1.0$$ 

The context vector for time step $i$ is then a weighted sum of the encoder hidden states (the *values*):
$$\mathbf{c}_i = \sum_j \alpha_{ij} \mathbf{h}_j$$

In [71]:
class BahdanauAttention(nn.Module):
    """Implements Bahdanau (MLP) attention"""
    
    def __init__(self, hidden_size, key_size=None, query_size=None):
        super(BahdanauAttention, self).__init__()
        
        # We assume a bi-directional encoder so key_size is 2*hidden_size
        key_size = 2 * hidden_size if key_size is None else key_size
        query_size = hidden_size if query_size is None else query_size

        self.key_layer = nn.Linear(key_size, hidden_size, bias=False)
        self.query_layer = nn.Linear(query_size, hidden_size, bias=False)
        self.energy_layer = nn.Linear(hidden_size, 1, bias=False)
        
        # to store attention scores
        self.alphas = None
        
    def forward(self, query=None, proj_key=None, value=None, mask=None):
        assert mask is not None, "mask is required"

        # We first project the query (the decoder state).
        # The projected keys (the encoder states) were already pre-computated.
        query = self.query_layer(query)
        
        # Calculate scores.
        scores = self.energy_layer(torch.tanh(query + proj_key))
        scores = scores.squeeze(2).unsqueeze(1)
        
        # Mask out invalid positions.
        # The mask marks valid positions so we invert it using `mask & 0`.
        scores.data.masked_fill_(mask == 0, -float('inf'))
        
        # Turn scores to probabilities.
        alphas = F.softmax(scores, dim=-1)
        self.alphas = alphas        
        
        # The context vector is the weighted sum of the values.
        context = torch.bmm(alphas, value)
        
        # context shape: [B, 1, 2D], alphas shape: [B, 1, M]
        return context, alphas

## Embeddings and Softmax                                                                                                                                                                                                                                                                                           
We use learned embeddings to convert the input tokens and output tokens to vectors of dimension `emb_size`.

We will simply use PyTorch's [nn.Embedding](https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding) class.

## Full Model

Here we define a function from hyperparameters to a full model. 

In [72]:
def make_model(src_vocab, tgt_vocab, emb_size=100, hidden_size=100, num_layers=1, dropout=0.1):
    "Helper: Construct a model from hyperparameters."

    attention = BahdanauAttention(hidden_size)

    model = EncoderDecoder(
        Encoder(emb_size, hidden_size, num_layers=num_layers, dropout=dropout),
        Decoder(emb_size, hidden_size, attention, num_layers=num_layers, dropout=dropout),
        nn.Embedding(src_vocab, emb_size),
        nn.Embedding(tgt_vocab, emb_size),
        Generator(hidden_size, tgt_vocab))

    return model.cuda(device=1) if USE_CUDA else model

# Training

This section describes the training regime for our models.

We stop for a quick interlude to introduce some of the tools 
needed to train a standard encoder decoder model. First we define a batch object that holds the src and target sentences for training, as well as their lengths and masks. 

## Batches and Masking

In [73]:
class Batch:
    """Object for holding a batch of data with mask during training.
    Input is a batch from a torch text iterator.
    """
    def __init__(self, src, trg, pad_index=0):
        
        src, src_lengths = src
        
        self.src = src
        self.src_lengths = src_lengths
        self.src_mask = (src != pad_index).unsqueeze(-2)
        self.nseqs = src.size(0)
        
        self.trg = None
        self.trg_y = None
        self.trg_mask = None
        self.trg_lengths = None
        self.ntokens = None

        if trg is not None:
            trg, trg_lengths = trg
#             self.trg = trg[:, :-1]
            self.trg = trg
            self.trg_lengths = trg_lengths
            self.trg_y = trg[:, 1:]
            self.trg_mask = (self.trg_y != pad_index)
            self.ntokens = (self.trg_y != pad_index).data.sum().item()
        
        if USE_CUDA:
            self.src = self.src.cuda(device=1)
            self.src_mask = self.src_mask.cuda(device=1)

            if trg is not None:
                self.trg = self.trg.cuda(device=1)
                self.trg_y = self.trg_y.cuda(device=1)
                self.trg_mask = self.trg_mask.cuda(device=1)
                

## Training Loop
The code below trains the model for 1 epoch (=1 pass through the training data).

In [74]:
def run_epoch(data_iter, model, loss_compute, print_every=50):
    """Standard Training and Logging Function"""

    start = time.time()
    total_tokens = 0
    total_loss = 0
    print_tokens = 0

    for i, batch in enumerate(data_iter, 1):
#         print(batch.src.size())
#         print(batch.trg.size())
#         print(batch.src_mask.size())
#         print(batch.trg_mask.size())
#         print(len(batch.src_lengths))
#         print(len(batch.trg_lengths))
        out, _, pre_output = model.forward(batch.src, batch.trg,
                                           batch.src_mask, batch.trg_mask,
                                           batch.src_lengths, batch.trg_lengths)
#         print(out.size())
#         print(pre_output.size())
#         print(batch.trg_y.size())
#         print(batch.nseqs)
        loss = loss_compute(pre_output, batch.trg_y, batch.nseqs)
        total_loss += loss
        total_tokens += batch.ntokens
        print_tokens += batch.ntokens
        
        if model.training and i % print_every == 0:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.nseqs, print_tokens / elapsed))
            start = time.time()
            print_tokens = 0

    return math.exp(total_loss / float(total_tokens))

## Training Data and Batching

We will use torch text for batching. This is discussed in more detail below. 

## Optimizer

We will use the [Adam optimizer](https://arxiv.org/abs/1412.6980) with default settings ($\beta_1=0.9$, $\beta_2=0.999$ and $\epsilon=10^{-8}$).

We will use $0.0003$ as the learning rate here, but for different problems another learning rate may be more appropriate. You will have to tune that.

# A First  Example

We can begin by trying out a simple copy-task. Given a random set of input symbols from a small vocabulary, the goal is to generate back those same symbols. 

## Synthetic Data

In [75]:
def data_gen(num_words=11, batch_size=16, num_batches=100, length=10, pad_index=0, sos_index=1):
    """Generate random data for a src-tgt copy task."""
    for i in range(num_batches):
        data = torch.from_numpy(
          np.random.randint(1, num_words, size=(batch_size, length)))
        data[:, 0] = sos_index
        data = data.cuda(device=1) if USE_CUDA else data
        src = data[:, 1:]
        trg = data
        src_lengths = [length-1] * batch_size
        trg_lengths = [length] * batch_size
        yield Batch((src, src_lengths), (trg, trg_lengths), pad_index=pad_index)

## Loss Computation

In [76]:
class SimpleLossCompute:
    """A simple loss compute and train function."""

    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt

    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)),
                              y.contiguous().view(-1))
        loss = loss / norm
#         print('Inside loss call', loss.item())
#         print(x.size(), y.size())

        if self.opt is not None:
            loss.backward()          
            self.opt.step()
            self.opt.zero_grad()

        return loss.data.item() * norm

## Training the copy task

In [77]:
def train_copy_task():
    """Train the simple copy task."""
    num_words = len(word_to_index) + 2
#     num_words = len(brown.words()) + 2
#     criterion = nn.NLLLoss(reduction="sum", ignore_index=0)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    model = make_model(num_words, num_words, emb_size=300, hidden_size=64)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    #eval_data = list(data_gen1(num_words=num_words, batch_size=1, num_batches=100))#Batch object
    dev_perplexities = []
    
    if USE_CUDA:
        model.cuda(device=1)

    for epoch in range(20):
        
        print("Epoch %d" % epoch)

        # train
        model.train()
        data = data_gen1(list_of_sents_src_train, list_of_sents_trg_train, num_words=num_words, batch_size=16, num_batches=100)
        trained_perplexity= run_epoch(data, model,
                  SimpleLossCompute(model.generator, criterion, optim))
        print(trained_perplexity)
    return model  

        # evaluate
#         model.eval()
#         with torch.no_grad(): 
#             perplexity = run_epoch(eval_data, model,
#                                    SimpleLossCompute(model.generator, criterion, None))
#             print("Evaluation perplexity: %f" % perplexity)
#             dev_perplexities.append(perplexity)
#             print_examples(eval_data, model, n=2, max_len=9)
      
#     return dev_perplexities

In [78]:
model = train_copy_task()
# data = data_gen1(num_words=num_words, batch_size=32, num_batches=100)

/home/development/kevin/installations/anaconda3/lib/python3.5/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 0
Epoch Step: 50 Loss: 0.271263 Tokens per Sec: 2718.308029
Epoch Step: 100 Loss: 0.157172 Tokens per Sec: 2713.030990
1.019893199833318
Epoch 1
Epoch Step: 50 Loss: 0.044570 Tokens per Sec: 3269.053057
Epoch Step: 100 Loss: 0.015651 Tokens per Sec: 2908.837636
1.003820795536479
Epoch 2
Epoch Step: 50 Loss: 0.007457 Tokens per Sec: 3200.240146
Epoch Step: 100 Loss: 0.004152 Tokens per Sec: 2931.833668
1.0008570813023527
Epoch 3
Epoch Step: 50 Loss: 0.003159 Tokens per Sec: 2708.660193
Epoch Step: 100 Loss: 0.002183 Tokens per Sec: 3191.661970
1.0004910797209092
Epoch 4
Epoch Step: 50 Loss: 0.001611 Tokens per Sec: 2984.421200
Epoch Step: 100 Loss: 0.001031 Tokens per Sec: 2795.490860
1.0003681212902642
Epoch 5
Epoch Step: 50 Loss: 0.000857 Tokens per Sec: 3224.975261
Epoch Step: 100 Loss: 0.000612 Tokens per Sec: 3313.418959
1.000296462012745
Epoch 6
Epoch Step: 50 Loss: 0.000658 Tokens per Sec: 3359.890601
Epoch Step: 100 Loss: 0.000439 Tokens per Sec: 3303.899605
1.000270209665

In [79]:
def get_argmax(probs, input_list):
#     print(probs)
    max_val = -np.Inf
    max_candidate = None
    for index in input_list:
        if max_val < probs[index]:
            max_val = probs[index]
            max_candidate = index
    return max_candidate

In [80]:
# Checking training example

In [81]:
num_words = len(word_to_index) + 2
data = data_gen1(list_of_sents_src_train, list_of_sents_trg_train, num_words=num_words, batch_size=32, num_batches=100)
# model()
for i, batch in enumerate(data, 1):
    out, _, pre_output = model.forward(batch.src, batch.trg,
                                           batch.src_mask, batch.trg_mask,
                                           batch.src_lengths, batch.trg_lengths)
    break

In [82]:
pre_output_vocab = model.generator(pre_output)

In [83]:
pre_output_vocab.size()

torch.Size([32, 30, 56059])

In [84]:
pre_output_vocab.size()

torch.Size([32, 30, 56059])

In [85]:
probs = pre_output_vocab[0,1,:].cpu().detach().numpy().tolist()#first sent, 2nd word, all prob distribution.
input_list = batch.trg[5].cpu().detach().numpy().tolist()
input_words = []
for w in input_list:
    input_words.append(index_to_word[w-2])
print(input_words,len(input_words))

['Belleville', 'If', 'any', 'among', 'the', 'hardy', 'hundreds', 'who', 'sat', 'in', 'the', 'downpour', 'are', 'in', 'doubt', 'about', 'how', 'it', 'comes', 'out', ',', 'let', 'them', 'take', 'comfort', '.', 'Basement', 'Basement', 'Basement', 'Basement', 'Basement'] 31


In [86]:
probs = pre_output_vocab[0,1,:].cpu().detach().numpy().tolist()#first sent, 2nd word, all prob distribution.
input_list = batch.src[5].cpu().detach().numpy().tolist()
input_words = []
for w in input_list:
    input_words.append(index_to_word[w-2])
print(input_words,len(input_words))

[',', 'If', 'about', 'among', 'any', 'are', 'comes', 'comfort', 'doubt', 'downpour', 'hardy', 'how', 'hundreds', 'in', 'in', 'it', 'let', 'out', 'sat', 'take', 'the', 'the', 'them', 'who', '.', 'Basement', 'Basement', 'Basement', 'Basement', 'Basement'] 30


In [88]:
input_list = batch.src[5].cpu().detach().numpy().tolist()
for i in range(30):
    probs =pre_output_vocab[5,i,:].cpu().detach().numpy().tolist()
    cur_candidate = get_argmax(probs, input_list)
    print(index_to_word[cur_candidate-2], end=' ')
    input_list.remove(cur_candidate)

If any among the hardy hundreds who sat in the downpour are in doubt about how it comes out , let them take comfort . Basement Basement Basement Basement Basement 

In [77]:
# Checking test example

In [54]:
num_words = len(word_to_index) + 2
data = data_gen1(list_of_sents_src_test, list_of_sents_trg_test, num_words=num_words, batch_size=32, num_batches=100)
# model()
for i, batch in enumerate(data, 1):
    out, _, pre_output = model.forward(batch.src, batch.trg,
                                           batch.src_mask, batch.trg_mask,
                                           batch.src_lengths, batch.trg_lengths)
    break

In [55]:
pre_output_vocab = model.generator(pre_output)

In [56]:
pre_output_vocab.size()

torch.Size([32, 30, 56059])

In [29]:
pre_output_vocab.size()

torch.Size([32, 30, 56059])

In [38]:
probs = pre_output_vocab[0,1,:].cpu().detach().numpy().tolist()#first sent, 2nd word, all prob distribution.
input_list = batch.trg[5].cpu().detach().numpy().tolist()
input_words = []
for w in input_list:
    input_words.append(index_to_word[w-2])
print(input_words,len(input_words))

['Belleville', 'Only', 'in', 'one', 'instance', 'have', 'we', 'moved', 'purposively', 'and', 'effectively', 'to', 'dislodge', 'existing', 'Communist', 'power', ':', 'in', 'Guatemala', '.', 'Basement', 'Basement', 'Basement', 'Basement', 'Basement', 'Basement'] 26


In [39]:
probs = pre_output_vocab[0,1,:].cpu().detach().numpy().tolist()#first sent, 2nd word, all prob distribution.
input_list = batch.src[5].cpu().detach().numpy().tolist()
input_words = []
for w in input_list:
    input_words.append(index_to_word[w-2])
print(input_words,len(input_words))

[':', 'Communist', 'Guatemala', 'Only', 'and', 'dislodge', 'effectively', 'existing', 'have', 'in', 'in', 'instance', 'moved', 'one', 'power', 'purposively', 'to', 'we', '.', 'Basement', 'Basement', 'Basement', 'Basement', 'Basement', 'Basement'] 25


In [50]:
input_list = batch.src[5].cpu().detach().numpy().tolist()
for i in range(26):
    probs = pre_output_vocab[0,i,:].cpu().detach().numpy().tolist()
    cur_candidate = get_argmax(probs, input_list)
    print(index_to_word[cur_candidate-2], end=' ')
    input_list.remove(cur_candidate)

Only effectively in in we moved and . to have : Guatemala Communist purposively existing dislodge instance one power Basement Basement Basement Basement Basement Basement 

IndexError: index 25 is out of bounds for dimension 0 with size 25

In [ ]:
print(get_argmax(pre_output_vocab[0,1,:].cpu().detach().numpy().tolist(), batch.trg[0].cpu().detach().numpy().tolist()))

In [ ]:
for i in range(0, 10):
    print(get_argmax(pre_output_vocab[0,i,:].cpu().detach().numpy().tolist(), batch.trg[0].cpu().detach().numpy().tolist()))

## create files

In [92]:
num_words = len(word_to_index) + 2
data = data_gen1(list_of_sents_src_train, list_of_sents_trg_train, num_words=num_words, batch_size=32, num_batches=100)
# model()
for i, batch in enumerate(data, 1):
    out, _, pre_output = model.forward(batch.src, batch.trg,
                                           batch.src_mask, batch.trg_mask,
                                           batch.src_lengths, batch.trg_lengths)
    pre_output_vocab = model.generator(pre_output)
    for j in range(len(batch.trg)):
        input_list = batch.trg[j].cpu().detach().numpy().tolist()
        input_words = ''
        for w in input_list:
            input_words +=index_to_word[w-2]+' '
        file_name = 'results/target_op/'+'batch_'+str(i)+'_sent_'+str(j)+'.txt'
        f = open(file_name, "w")
        f.write(input_words)
        f.close()
        
        input_list = batch.src[j].cpu().detach().numpy().tolist()
        input_words = ''
        for w in input_list:
            input_words +=index_to_word[w-2]+' '
        file_name = 'results/input/'+'batch_'+str(i)+'_sent_'+str(j)+'.txt'
        f = open(file_name, "w")
        f.write(input_words)
        f.close()
        
        input_list = batch.src[j].cpu().detach().numpy().tolist()
        input_words = ''
        for k in range(29):
            probs = pre_output_vocab[j,k,:].cpu().detach().numpy().tolist()
            cur_candidate = get_argmax(probs, input_list)
            input_words += index_to_word[cur_candidate-2] + ' '
            #print(index_to_word[cur_candidate-2], end=' ')
            input_list.remove(cur_candidate)
        file_name = 'results/actual_op/'+'batch_'+str(i)+'_sent_'+str(j)+'.txt'
        f = open(file_name, "w")
        f.write(input_words)
        f.close()